# Environment setup

This is a frame of the `main` notebook. It contains code to set up the environment and load the LLama pipeline.

# Installations

Before we proceed, we need to ensure that the essential libraries are installed:
- `Hugging Face Transformers`: Provides us with a straightforward way to use pre-trained models and datasets.
- `PyTorch`: Serves as the backbone for deep learning operations.
- `Accelerate`: Optimizes PyTorch operations, especially on GPU.
- `rouge_score`: Allows us to get the ROUGE metric on Python.
- `datasets`: Allows us to load datasets from the transformer library.
- `py7zr`: Required in order to use the SAMSum corpus.

In [ ]:
!pip install requirements.txt

# Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.



In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: Traceback (most recent call last):
  File "/opt/mamba/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/opt/mamba/lib/python3.10/site-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    service.run()
  File "/opt/mamba/lib/python3.10/site-packages/huggingface_hub/comma

In [4]:
!huggingface-cli whoami

linogaliana


#I. Creation of the work environment

# I.1. Loading Dataset

We are going to analyze the dataset and see how we could use it to best fit our purpose.

First, let's load the dataset and see how it looks like.

In [1]:
from datasets import load_dataset

dataset = load_dataset("samsum")

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

Now we know that we have three subdatasets: `train`, `test`, `validation`.

It will come in handy to train and evaluate the fine-tuned model but let's look a little bit further and see how a row looks like.  

In [2]:
def show_row(dataset):
  """
  Show the first row of the dataset.

  Parameters:
        dataset (Dataset): The dataset which will be used to train the AI.

  Returns:
      None: Prints the row.
  """
  sample = dataset['train'].select(range(1))
  for line in sample:
    print('Row:',  line, '\n')
    print(line['dialogue'], '\n')
    print('>> ', line['summary'], '\n')

show_row(dataset)

Row: {'id': '13818513', 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)", 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'} 

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-) 

>>  Amanda baked cookies and will bring Jerry some tomorrow. 



The summary is a third-person utterance. However we want to train our model to summarize the message in the first-person to make it more conversational.

Also the corpus contains around 25% of dialogues with more than two characters in it. We want to get rid of them because it will be harder to transform it to a first-person perspective with an AI model.

We will be using Meta's model `LLama2` as it is a top-notch open source model handling text generation.

# I.2. Loading Model & Tokenizer

Here, we are preparing our session by loading both the Llama model and its associated tokenizer.

The tokenizer will help in converting our text prompts into a format that the model can understand and process.

In [6]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.

# I.3. Creating the Llama Pipeline

We'll set up a pipeline for text generation.

This pipeline simplifies the process of feeding prompts to our model and receiving generated text as output.

*Note*: This cell takes 2-3 minutes to run

In [ ]:
from transformers import pipeline
import accelerate

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
def get_llama_response(prompts):
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompts,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )

    return [sequence[0]['generated_text'] for sequence in sequences]

We will be working with the following prompt:

*Transform the following third-person sentence into first-person. Replace `Name1`'s pronouns and possessive determiners with first-person counterparts. Replace `Name2`'s pronouns and possessive determiners with second-person counterparts.  Adjust verb forms accordingly. Write the transformed sentence only and write this mention at the begining: 'Answer:'. '`Summary`'*

Let's try it out with the example above.  

In [ ]:
prompt = "Transform the following third-person sentence into first-person. Replace Amanda's pronouns and possessive determiners with first-person counterparts. Replace Jerry's pronouns and possessive determiners with second-person counterparts. Adjust verb forms accordingly. Write the transformed sentence only and write this mention at the begining: 'Answer:'. 'Amanda baked cookies and will bring Jerry some tomorrow. '"
responses = get_llama_response([prompt])

print(responses[0])

You might not get a satisfying response on your first attempt but if you retry a few attemps you should get something like this:

*Answer: I baked cookies and will bring you some tomorrow.*

Here the LLama's response is satisfying. However we are going to introduce a metric later in order to evaluate the performance of the model.